# Title
[]()

In [1]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from summary_chain import *
from response_processing import *
from article_processing import create_text_dict_from_folder
from file_functions import *
import time

# Create text dictionary
folder_path = '../text/2023-06-02 1' # ** UPDATE REQUIRED**

encoding='ISO-8859-1'
subset=None

text_dict = create_text_dict_from_folder(folder_path, encoding=encoding, subset=subset)

# Set parameters
iteration_id = 1
n_choices = 5
pause_per_request=0
summary_iteration_id = iteration_id
chatbot_id = iteration_id

# Create prompt lists
prep_step = [
    "Take the key points and numerical descriptors to",
]

summarize_task = [
    "Summarize for a LinkedIn post.",
    # "Describe the interesting points to your coworker at the water cooler",
    # "Create an Instagram post without hashtags.",
]

user_simplify_task = [
    """If needed, rewrite the text using terms appropriate for the audience. If not keep it the same.\
    Follow these steps to accomplish this: \
    \n1. Check if the content and language are appropriate for the audience. \
    \n2. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n3. Return the final version of the summary to be shown to the audience. \
    \n\nYour audience is""",
]

simplify_audience = [
    # "a lay audience",
    "people without a science background",
]

user_relevance_task = [
    """Rewrite this summary to include a statement of how it is relevant for the audience. \
        Follow these steps to accomplish this: \
        \n1. Think about why this might be relevant for the audience in the grand scheme of things.\
        \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, \
        add a sentence to inform the audience. Otherwise, keep it the same. \
        \n3. Modify the summary if needed to reduce redundancy. \
        \n4. Check if the content and language are appropriate for the audience. \
        If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
        \n5. Return the final version of the summary to be shown to the audience. \
        \n6. Remove the backticks.
        \n\nYour audience consists of""",
]

relevance_audience = [
    # "seniors",
    "people who enjoy sports",
    # "people new to resistance training"
]

qna_dict = dict()
chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()
chain_results_dict = dict()

# Create initial summaries
qna_dict, chaining_dict = batch_summarize_chain(
    text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id
    )

time.sleep(pause_per_request)

# Create simple summaries
simplify_prompts = user_simplify_task
audience = simplify_audience
simple_summaries = prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, 
    chaining_dict[iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

# Add relevance
relevance_prompts = user_relevance_task
relevance = prompt_chaining_dict(relevance_prompts, relevance_audience, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id, prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

# Merge the results
try:
    chain_results_dict = merge_all_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, pivot=True,
        empty_columns=True,
        save_df=True, save_chatbot=True, 
            csv_path=folder_path,
    )
    print(f'\nCompleted merge_all_chaining_results!:)')
except:
    merged_df = merge_chaining_results(
        qna_dict, chatbot_dict, 
        simple_summaries_dict, relevance_dict, iteration_id, 
        empty_columns=True, pivot=True, validate=True, 
        chatbot_id=chatbot_id, save_df=True, save_chatbot=True,
        csv_path=folder_path, pickle_path=folder_path, json_path=folder_path
        )
    print(f'\nCompleted merge_chaining_results.')

**Text #1 prompt #0 of 0**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 5 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
DataFrame shape: (5, 9)
summary_iteration_id: 1
n_choices: 1
**text1_prompt00
simplify_iteration:  1
Task: If needed, rewrite the text using terms appropriate for the audience. If not keep it the same.    Follow these steps to accomplish this:     
1. Check if the content and language are appropriate for the audience.     
2. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    
3. Return the final version of the summary to be shown to the audience.     

Your audience is people without a science background
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04', 'response_05']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requestin

In [2]:
merged_df

,B: article_title,C: choice,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,...,T: simple summary language rating,U: top simple summary,V: relevance task,W: full add relevance task,X: people who enjoy sports,Y: added relevance content rating,Z: added relevance language rating,AA: top added relevance,AB: add relevance task (seniors),AC: full add relevance task (seniors)
0,A step-defined sedentary lifestyle index 5000 ...,1,You are an expert at science communication.,gpt-3.5-turbo,A step-defined sedentary lifestyle index 5000 ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,Summary:\nStudies have used <5000 steps/day to...,,...,,,Rewrite this summary to include a statement of...,,"Summary:\nFor sports enthusiasts, it is import...",,,,,
1,A step-defined sedentary lifestyle index 5000 ...,2,You are an expert at science communication.,gpt-3.5-turbo,A step-defined sedentary lifestyle index 5000 ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,Summary: Studies have used 5000 steps/day as a...,,...,,,Rewrite this summary to include a statement of...,,Regular physical activity is important for mai...,,,,,
2,A step-defined sedentary lifestyle index 5000 ...,3,You are an expert at science communication.,gpt-3.5-turbo,A step-defined sedentary lifestyle index 5000 ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,"""Studies show that taking fewer than 5000 step...",,...,,,Rewrite this summary to include a statement of...,,Regular physical activity is essential for mai...,,,,,
3,A step-defined sedentary lifestyle index 5000 ...,4,You are an expert at science communication.,gpt-3.5-turbo,A step-defined sedentary lifestyle index 5000 ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,"""New research suggests that taking less than 5...",,...,,,Rewrite this summary to include a statement of...,,New research suggests that taking less than 50...,,,,,
4,A step-defined sedentary lifestyle index 5000 ...,5,You are an expert at science communication.,gpt-3.5-turbo,A step-defined sedentary lifestyle index 5000 ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,"""Studies suggest that individuals taking fewer...",,...,,,Rewrite this summary to include a statement of...,,Regular physical activity is essential for mai...,,,,,


# *End of Page*